#### Threat analysis possed by top 50 delegates of ENS

**FAQ:** Why only top 50? <br>
<code>$ </code>It is not hard-coded. One can provide a list of as many delegates as they want to the script. At the current moment however, top 50 delegates combine to make up just over 68% of the total voting power of ENS DAO. This is sufficient to achieve super-majority. It makes sense to make the cut-off at 50 at the time being.

In [20]:
# Cell 1: environment

import matplotlib.font_manager as fm
import numpy as np
import random as rand
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib.pyplot import gca
import os 
import math as mt
from bs4 import BeautifulSoup
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker
import pandas as pd
import requests
import scipy.fftpack
from pprint import pprint
%matplotlib inline

cwd = os.getcwd()
pwd = os.path.abspath(os.path.join(cwd, os.pardir))
fontloc = pwd + '/SF-Mono/SFMono-Regular.otf'
font = fm.FontProperties(fname = fontloc,size = 8); prop = font
proplr = fm.FontProperties(fname = fontloc,size = 12)
prop10 = fm.FontProperties(fname = fontloc,size = 10)

fontlocit = pwd + '/SF-Mono/SFMono-RegularItalic.otf'
fontit = fm.FontProperties(fname = fontlocit,size = 8)

#ticks font
def ticks(ax, size):
    font = fm.FontProperties(fname = fontloc,size = size)
    for label in ax.get_xticklabels():
        label.set_fontproperties(font)
    for label in ax.get_yticklabels():
        label.set_fontproperties(font)
        
def ticks3(ax, x, y, z):
    font = fm.FontProperties(fname = fontloc,size = x)
    for label in ax.get_xticklabels():
        label.set_fontproperties(font)
    font = fm.FontProperties(fname = fontloc,size = y)
    for label in ax.get_yticklabels():
        label.set_fontproperties(font)
    font = fm.FontProperties(fname = fontloc,size = z)
    for label in ax.get_zticklabels():
        label.set_fontproperties(font)
        
def xticks(ax, size):
    font = fm.FontProperties(fname = fontloc,size = size)
    for label in ax.get_xticklabels():
        label.set_fontproperties(font)
        
def yticks(ax, size):
    font = fm.FontProperties(fname = fontloc,size = size)
    for label in ax.get_yticklabels():
        label.set_fontproperties(font)

@ticker.FuncFormatter
def major_formatter(x, pos):
    label = str("{0:.1f}".format(x)) if x < 0 else str("{0:.2f}".format(x))
    return label

import matplotlib.colors as colors

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap


In [21]:
# read in data
file_in = cwd + '/ENS.log'
f = open(file_in, 'r')
lines = f.readlines()[1:]
f.close()

rank = []
perc = []
vote = []
name = []

for line in lines:
    if not line.startswith("#"):
        column = line.split()
        rank.append(int(column[0]))
        name.append(str(column[1]))
        perc.append(float(column[2]))
        vote.append(int(column[3]))
            
lines = []
voteCache = vote; percCache = perc

#### Utopia

Consider the hypothetical utopic scenario first. In the trivial utopic scenario where the votes are equally distributed among the delegates following a flat power-spectrum, the analytical estimation of attack surface becomes trivial as well. 

In [22]:
%config InlineBackend.figure_format = 'svg'
import itertools

totalVotes = int(np.sum(vote)/(np.sum(perc)*0.01))
print('Total Castable Votes: ' + str(totalVotes))
threshold = 1000000
Total = len(rank)

#N-party attack in Utopia
vote = [totalVotes/Total for i in range(Total)]
perc = [100.0/Total for i in range(Total)]
print('Utopia: ' + str(int(totalVotes/Total)) + ' @ ' + str(100.0/Total) + '%')

N = 5; possVec = []
for perps in range(1, Total):
    print('Possible Collusions: ' + str(int(np.math.factorial(Total)/(np.math.factorial(Total - perps) * np.math.factorial(perps)))))
    possVec.append(int(np.math.factorial(Total)/(np.math.factorial(Total - perps) * np.math.factorial(perps))))

print('#Threat Analysis in Utopia')
print('#colluders', '#possibilities', '#totalPossibilities')
for perps in range(3,N + 1):
    perms = list(itertools.combinations(rank,perps))
    threat = []; attackers = []
    for val in perms:
      attack = 0; names = ''; ranks = []
      for i in val:
          attack = attack + vote[int(i - 1)]
          #names = names + name[int(i - 1)] + ';'
          ranks.append(rank[int(i - 1)])
      if attack > threshold:
           #attackers.append(names)
           threat.append(ranks)
      attack = 0
      

    print(perps, len(threat), len(perms))
    
perc = []; vote = []



Total Castable Votes: 6847327
Utopia: 136946 @ 2.0%
Possible Collusions: 50
Possible Collusions: 1225
Possible Collusions: 19600
Possible Collusions: 230300
Possible Collusions: 2118760
Possible Collusions: 15890700
Possible Collusions: 99884400
Possible Collusions: 536878650
Possible Collusions: 2505433700
Possible Collusions: 10272278170
Possible Collusions: 37353738800
Possible Collusions: 121399651100
Possible Collusions: 354860518600
Possible Collusions: 937845656300
Possible Collusions: 2250829575120
Possible Collusions: 4923689695575
Possible Collusions: 9847379391150
Possible Collusions: 18053528883775
Possible Collusions: 30405943383200
Possible Collusions: 47129212243960
Possible Collusions: 67327446062800
Possible Collusions: 88749815264600
Possible Collusions: 108043253365600
Possible Collusions: 121548660036300
Possible Collusions: 126410606437752
Possible Collusions: 121548660036300
Possible Collusions: 108043253365600
Possible Collusions: 88749815264600
Possible Collusio

#### Real World

File [ENS.log](https://github.com/inplco/dao-attacks/blob/master/ENS/ENS.log) contains voting data of top 50 ENS delegates scrapped from [Sybil.org](https://sybil.org). The file header contains four columns: <br>
<code>#</code>rank of the delegate, <br>
<code>#</code>name of delegate, <br>
<code>#</code>voting share in percentage of the delegate, and <br>
<code>#</code>number of votes of the delegate.

In [23]:
#N-party attack in reality
print('#Threat Analysis in Real World')
print('#colluders', '#possibilities', '#totalPossibilities')
perc = percCache; vote = voteCache
for perps in range(3,N + 1):
    perms = list(itertools.combinations(rank,perps))
    threat = []; attackers = []
    for val in perms:
      attack = 0; names = ''; ranks = []
      for i in val:
          attack = attack + vote[int(i - 1)]
          #names = names + name[int(i - 1)] + ';'
          ranks.append(rank[int(i - 1)])
      
      if attack > threshold:
           #attackers.append(names)
           threat.append(ranks)
      attack = 0

    print(perps, len(threat), len(perms))

#Threat Analysis in Real World
#colluders #possibilities #totalPossibilities
3 0 19600
4 302 230300
5 17920 2118760
